## USING VECTOR SPACE MODEL AND COSINE SIMILARITY FOR TEXT CLASSIFICATION

### CONSTRUCT THE VECTOR SPACE MODEL (PREPROCESS THE TEXT, CALCULATE THE BAG OF WORDS AND TF-IDF) AND COMPUTE THE IMPORTANCE OF THE WORD CHINESE IN THE TEST DATA

In [ ]:
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np

In [ ]:
train_documents = [
    "Chinese Beijing Chinese",
    "Chinese Chinese Shanghai",
    "Chinese Macao",
    "Tokyo Japan Chinese"
]

test_document = "Chinese Chinese Chinese Tokyo Japan"

In [ ]:
vectorizer_bow = CountVectorizer()
X_bow = vectorizer_bow.fit_transform(train_documents)
vectorizer_tfidf = TfidfVectorizer()
X_tfidf = vectorizer_tfidf.fit_transform(train_documents)

In [ ]:
test_vector_tfidf = vectorizer_tfidf.transform([test_document])
importance_of_chinese = test_vector_tfidf[0, vectorizer_tfidf.vocabulary_["chinese"]]

In [ ]:
print("Importance of the word 'Chinese' in the test data (TF-IDF):", importance_of_chinese)

Importance of the word 'Chinese' in the test data (TF-IDF): 0.7420574954436144


### FIND THE SIMILARITY OF THE TEST DATA CONSIDERING WITH ANY ONE DOCUMENT FROM TRAINING DATA USING THE COSINE SIMILARITY EVALUATION METRIC

In [ ]:
X_tfidf_dense = tf.convert_to_tensor(X_tfidf.toarray(), dtype=tf.float32)
test_vector_tfidf_dense = tf.convert_to_tensor(test_vector_tfidf.toarray(), dtype=tf.float32)

In [ ]:
selected_document_tfidf = X_tfidf_dense[0]

In [ ]:
dot_product = tf.reduce_sum(tf.multiply(test_vector_tfidf_dense, selected_document_tfidf), axis=1)
magnitude_test = tf.sqrt(tf.reduce_sum(tf.square(test_vector_tfidf_dense), axis=1))
magnitude_doc = tf.sqrt(tf.reduce_sum(tf.square(selected_document_tfidf)))

In [ ]:
cosine_similarity = dot_product / (magnitude_test * magnitude_doc)
cosine_similarity_value = cosine_similarity.numpy()[0]

In [ ]:
print("Cosine Similarity between test data and selected training document:", cosine_similarity_value)

Cosine Similarity between test data and selected training document: 0.5358071


### TAKE A DATASET OF YOUR OWN LABELLED WITH SENTIMENT. SPLIT THE TRAINING AND TESTING PART AND COMPUTE THE SENTIMENT CLASSIFICATION WITH THE APPLICATION OF LAPLACE SMOOTHING

In [ ]:
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

In [ ]:
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_data, test_data = dataset['train'], dataset['test']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.YW4XHH_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.YW4XHH_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.YW4XHH_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_sentences = []
train_labels = []
for sentence, label in tfds.as_numpy(train_data):
    train_sentences.append(str(sentence))
    train_labels.append(int(label))

In [ ]:
test_sentences = []
test_labels = []
for sentence, label in tfds.as_numpy(test_data):
    test_sentences.append(str(sentence))
    test_labels.append(int(label))

In [ ]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_sentences, train_labels, test_size=0.2, random_state=42
)

In [ ]:
vocab_size = 10000
max_length = 100
oov_tok = "<OOV>"

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)

In [ ]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

In [ ]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=16, input_length=max_length),
    Bidirectional(LSTM(32)),
    Dense(24, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.build(input_shape=(None, max_length))

In [ ]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 100, 16)             │         160,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_4 (Bidirectional)      │ (None, 64)                  │          12,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 24)                  │           1,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │              25 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 174,129 (680.19 KB)

 Trainable params: 174,129 (680.19 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(train_padded, np.array(train_labels), epochs=5, validation_data=(val_padded, np.array(val_labels)))

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 60s 89ms/step - accuracy: 0.5917 - loss: 0.6451 - val_accuracy: 0.7480 - val_loss: 0.5202
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 73s 75ms/step - accuracy: 0.8393 - loss: 0.4104 - val_accuracy: 0.7970 - val_loss: 0.4386
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 80s 71ms/step - accuracy: 0.8664 - loss: 0.3306 - val_accuracy: 0.8320 - val_loss: 0.3917
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 71ms/step - accuracy: 0.9050 - loss: 0.2462 - val_accuracy: 0.8344 - val_loss: 0.3770
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 70ms/step - accuracy: 0.9355 - loss: 0.1839 - val_accuracy: 0.8418 - val_loss: 0.4216


In [ ]:
test_loss, test_acc = model.evaluate(test_padded, np.array(test_labels))
print("Test Accuracy:", test_acc)

782/782 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.8361 - loss: 0.4306
Test Accuracy: 0.8347200155258179
